In [62]:
import numpy as np
import os
import matplotlib.pyplot as plt
import h5py
from scipy import interpolate
import pandas as pd
import yaml
import geopandas as gp
import time
import pandas as pd
from geopy.distance import great_circle
import folium
import h5py
from scipy.signal import butter, lfilter

from scipy.spatial import ConvexHull
from scipy.interpolate import griddata
from shapely.geometry import Point, Polygon
import csv

In [63]:
import sys
sys.path.append(r'../tools_F-J/')
from toollib_standard import maplib
from toollib_standard import mathlib
from toollib_standard import filelib
from toollib_standard import stacklib
from toollib_standard import plotlib

In [64]:
flag_project = 1

if flag_project == 0:
    file_project = 'a-project.yml'
elif flag_project == 1:
    file_project = 'a-project_repar.yml'
elif flag_project == 2:
    file_project = 'a-project_voro.yml'
    
with open(file_project, 'r', encoding='utf-8') as f:
    proj = yaml.load(f.read(), Loader=yaml.FullLoader)
name_project = proj['name']
#name_project = 'project_repartition_v4.0/output_repar_v9.5_01--10-16Hz/'

In [65]:
with open('0_config.yml', 'r', encoding='utf-8') as f:
    dir_config = yaml.load(f.read(), Loader=yaml.FullLoader)
dir_project_workspace = dir_config['dir_project_workspace']
dir_CC_workspace = dir_config['dir_CC_workspace']
print('dir_CC_workspace: ', dir_CC_workspace)
print('dir_project_workspace: ', dir_project_workspace)
dir_project = os.path.join(dir_project_workspace, name_project)
print('dir_project: ', dir_project)

dir_CC_workspace:  ./
dir_project_workspace:  ./
dir_project:  ./project_repartition_v4.0/output_repar_v9.5_02--10-16Hz/


In [66]:
filename = dir_project+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = dir_project+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary

In [67]:
dir_CC = dir_CC_workspace + info_basic['rdir_CC']
dir_ds = dir_project + info_basic['rdir_ds']
dir_disp = dir_project + info_basic['rdir_disp_model']
dir_partition = dir_project + info_basic['rdir_partition']
dir_disp_pick = dir_project + info_basic['rdir_disp_pick']
dir_inv = dir_project + info_basic['rdir_inv_BFGS']

In [68]:
dir_surf = dir_project + info_basic['rdir_image'] + 'paper_tomo/surf/'
dir_surf_input = dir_surf + 'input/'
dir_surf_grid = dir_surf + 'grid/'
dir_surf_output = dir_surf + 'output/'
dir_origin = dir_project + info_basic['rdir_image'] + 'paper_tomo/origin/'
dir_origin_input = dir_origin + 'input/'   
dir_origin_output = dir_origin + 'output/'
if not os.path.exists(dir_surf):
    os.makedirs(dir_surf)
if not os.path.exists(dir_surf_input):
    os.makedirs(dir_surf_input)
if not os.path.exists(dir_surf_grid):
    os.makedirs(dir_surf_grid)
if not os.path.exists(dir_surf_output):
    os.makedirs(dir_surf_output)
if not os.path.exists(dir_origin):
    os.makedirs(dir_origin) 
if not os.path.exists(dir_origin_input):
    os.makedirs(dir_origin_input)
if not os.path.exists(dir_origin_output):
    os.makedirs(dir_origin_output)

In [69]:
key_subworks = info_basic['key_subworks']

In [70]:
stalistname_all = info_basic['stalistname_all']
stainfo = pd.read_excel(stalistname_all)
stalist_all = stainfo['Station'].tolist() 
lat_stations_all =  stainfo['latitude'].tolist() 
lon_stations_all =  stainfo['longitude'].tolist() 
elevations_all =  stainfo['elevation'].tolist()

In [71]:
stations_partition = {}
lat_stations_partition = {}
lon_stations_partition = {}
lat_centroid_partition = []
lon_centroid_partition = []
for key in info_basic['key_subworks']:
    filepath = dir_partition + str(key) + '.txt'
    stations_this, lat_stations_this, lon_stations_this = np.loadtxt(filepath, dtype='str' , unpack=True)
    stations_partition[key] = stations_this
    lat_stations_partition[key] = lat_stations_this.astype(float)
    lon_stations_partition[key] = lon_stations_this.astype(float)
    lat_centroid_partition.append(np.mean(lat_stations_this.astype(float)))
    lon_centroid_partition.append(np.mean(lon_stations_this.astype(float)))

In [81]:
# all stations
file = dir_project + 'stations_all_trans.txt'
stations, x_stations_all, y_stations_all = np.loadtxt(file, dtype='str' , unpack=True)
x_stations_all = x_stations_all.astype(float)
y_stations_all = y_stations_all.astype(float)

In [80]:
inter_file = dir_inv + 'vs_inter.npz'
inter = np.load(inter_file, allow_pickle=True)
vs_fund = inter['vs_fund']
vs = inter['vs']
x_inter_in = inter['x']
y_inter_in = inter['y']
lon_inter_in = inter['lon']
lat_inter_in = inter['lat']
z = inter['depth']
dz = inter['dz']
N = inter['N']

### Generate csv for stalist

In [83]:
headers = ['stations','longitude','latitude','elevation','x','y']
filename = dir_surf_input + 'stations.csv'
with open(filename,'w',newline='') as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    for i in range(len(stalist_all)):
        f_csv.writerow([stalist_all[i],lon_stations_all[i],lat_stations_all[i],elevations_all[i],x_stations_all[i],y_stations_all[i]])

In [14]:
headers = ['partition','longitude','latitude']
filename = dir_surf_input + 'partition.csv'
with open(filename,'w',newline='') as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    for i in range(len(lon_centroid_partition)):
        f_csv.writerow([key_subworks[i],lon_centroid_partition[i],lat_centroid_partition[i]])

In [15]:
for i in range(N):
    depth = i*dz
    headers = ['x','y','longitude','latitude','depth','vs','vs_fund']
    filename = dir_surf_input + 'vs_'+str(depth)+'.csv'
    with open(filename,'w',newline='') as f:
        f_csv = csv.writer(f)
        f_csv.writerow(headers)
        for j in range(len(lon_inter_in)):
            f_csv.writerow([x_inter_in[j],y_inter_in[j],lon_inter_in[j],lat_inter_in[j],depth,vs[i,j],vs_fund[i,j]])

### Phase

In [16]:
f_list = [4,8,15]
phase = {}
for f in f_list:
    phase[f] = []
for key in info_basic['key_subworks']:
    disp = np.loadtxt(dir_disp + 'disp_model_'+key+'.txt')
    disp = disp[disp[:,2] == 0]
    for f in f_list:
        phase[f].append(disp[disp[:,0]==f][0][1])

In [17]:
headers = ['longitude','latitude','4Hz','8Hz','15Hz']
filename = dir_surf_input + 'phase.csv'
with open(filename,'w',newline='') as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    for i in range(len(lon_centroid_partition)):
        f_csv.writerow([lon_centroid_partition[i],lat_centroid_partition[i],phase[4][i],phase[8][i],phase[15][i]])

### Faults

In [18]:
# fault surface trace
faults = np.load(dir_project +'clark_faults_trans.npy', allow_pickle='TRUE').item()
fault_infer = np.load(dir_project +'clark_fault_inferred_trans.npy', allow_pickle='TRUE').item()

In [19]:
for i in range(len(faults)):
    headers = ['longitude','latitude']
    filename = dir_surf_input + 'fault_'+str(i)+'.csv'
    # 插值
    with open(filename,'w',newline='') as f:
        f_csv = csv.writer(f)
        f_csv.writerow(headers)
        for k in range(len(faults['clark'+str(i+1)]['x'])):
            f_csv.writerow([faults['clark'+str(i+1)]['x'][k],faults['clark'+str(i+1)]['y'][k]])

In [20]:
headers = ['longitude','latitude']
filename = dir_surf_input + 'fault_infer.csv'
with open(filename,'w',newline='') as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    for i in range(len(fault_infer['x'])):
        f_csv.writerow([fault_infer['x'][i],fault_infer['y'][i]])

### Faults 3D

In [21]:
for i in range(len(faults)):
    headers = ['x','y','z']
    filename = dir_origin_input + 'fault3d_'+str(i)+'.csv'
    with open(filename,'w',newline='') as f:
        f_csv = csv.writer(f)
        f_csv.writerow(headers)
        for k in range(len(faults['clark'+str(i+1)]['x'])):
            if faults['clark'+str(i+1)]['y'][k] >= -15 and faults['clark'+str(i+1)]['y'][k] <= 620: 
                f_csv.writerow([faults['clark'+str(i+1)]['x'][k],faults['clark'+str(i+1)]['y'][k],0])

In [22]:
headers = ['x','y','z']
filename = dir_origin_input + 'fault3d_infer.csv'
with open(filename,'w',newline='') as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    for i in range(len(fault_infer['x'])):
        f_csv.writerow([fault_infer['x'][i],fault_infer['y'][i],0])

### Vs Verticle

In [23]:
inter_file = dir_inv + 'vs_inter_verti.npz'
inter = np.load(inter_file, allow_pickle=True)

In [24]:
x_inter_verti = inter['x'].tolist()
y_inter_verti = inter['y'].tolist()
vs_verti = inter['vs'].tolist()
vs_fund_verti = inter['vs_fund'].tolist()

In [25]:
z = np.arange(0,21) * dz
for trace in vs_verti.keys():
    x = x_inter_verti[trace]
    headers = ['x','y','z','vs','vs_fund']
    filename = dir_origin_input + 'shallow_vs_verti_trace-'+str(trace)+'.csv'
    with open(filename,'w',newline='') as f:
        f_csv = csv.writer(f)
        f_csv.writerow(headers)
        for i in range(len(x)):
            for k in range(len(z)):
                f_csv.writerow([x[i],y_inter_verti[trace][0],z[k],vs_verti[trace][k][i],vs_fund_verti[trace][k][i]])

### Density

In [58]:
dense_partition_file = dir_project + 'density_partitions.npz'
dense_target_file = dir_project + 'density_targets.npz'
dense_partition = np.load(dense_partition_file, allow_pickle=True)
dense_target = np.load(dense_target_file, allow_pickle=True)

In [59]:
x = dense_partition['x']
y = dense_partition['y']
density_partition = dense_partition['density']
density_target = dense_target['density']

In [60]:
headers = ['x','y','density']
filename_partition = dir_surf_input + 'density_partition.csv'
filename_target = dir_surf_input + 'density_target.csv'
with open(filename_partition,'w',newline='') as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    for i in range(len(x)):
        f_csv.writerow([x[i],y[i],density_partition[i]])
with open(filename_target,'w',newline='') as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    for i in range(len(x)):
        f_csv.writerow([x[i],y[i],density_target[i]])